# End-to-End Data-Grounded Financial Chatbot

## ​‍♂️ Problem Statement
Build a chatbot that answers questions strictly from `holdings.csv` and `trades.csv` without using external knowledge. It supports:
1. **Direct Fact Retrieval**
2. **Aggregation** (Counts)
3. **Performance Comparison** (P&L)

If data is missing or ambiguous, it strictly returns: `Sorry can not find the answer`.

In [ ]:
import pandas as pd
import re
import numpy as np

# Configuration
HOLDINGS_PATH = 'holdings.csv'
TRADES_PATH = 'trades.csv'

## 1. Data Loading & Preprocessing

In [ ]:
def load_data():
    try:
        holdings = pd.read_csv(HOLDINGS_PATH)
        trades = pd.read_csv(TRADES_PATH)
        
        # Clean column names
        holdings.columns = holdings.columns.str.strip()
        trades.columns = trades.columns.str.strip()
        
        # Type conversion for numeric columns
        for col in ['Quantity', 'Price', 'Principal', 'PL_YTD', 'Qty']:
            if col in holdings.columns:
                holdings[col] = pd.to_numeric(holdings[col], errors='coerce')
            if col in trades.columns:
                trades[col] = pd.to_numeric(trades[col], errors='coerce')
                
        return holdings, trades
    except FileNotFoundError:
        print("Error: CSV files not found. Please ensure they are in the working directory.")
        return None, None

h_df, t_df = load_data()
if h_df is not None:
    print("Data Loaded Successfully")
    print("Holdings Columns:", h_df.columns.tolist())
    print("Trades Columns:", t_df.columns.tolist())

## 2. Intent Classification & Entity Parsing
This module interprets the natural language query. While a lightweight LLM could be used here to generate the JSON structure, we implement a robust **Rule-Based/Regex** approach for this demonstration to ensure deterministic, reproducible results without requiring an API key.

In [ ]:
def classify_and_parse(question):
    """
    Parses the question to extract intent and filters.
    Returns metadata dictionary.
    """
    q_lower = question.lower()
    
    intent_metadata = {
        "intent": "UNKNOWN",
        "filters": {},
        "table": "holdings", # Default
        "agg": None,
        "sort_col": None
    }
    
    # --- 1. Performance Questions ---
    # Keyword: "performed better", "profit & loss", "p&l"
    if any(k in q_lower for k in ["performed better", "profit", "loss", "p&l"]):
        intent_metadata["intent"] = "PERFORMANCE"
        intent_metadata["sort_col"] = "PL_YTD"
        intent_metadata["target_col"] = "PortfolioName"
        return intent_metadata
        
    # --- 2. Aggregation Questions (Counts) ---
    # Keyword: "total number", "how many"
    if any(k in q_lower for k in ["total number", "count", "how many"]):
        intent_metadata["intent"] = "AGGREGATION"
        intent_metadata["agg"] = "count"
        
        # Detect Entity (Trades or Holdings)
        if "trades" in q_lower:
            intent_metadata["table"] = "trades"
        else:
            intent_metadata["table"] = "holdings"
            
        # Extract Fund Name (heuristic: "for [Fund Name]")
        # We specifically look for the phrase after "for"
        match = re.search(r"for\s+(?:the\s+|a\s+)?([\w\s\-]+?)(?:$|\?|\sfund)", q_lower)
        if match:
            raw_ent = match.group(1).strip()
            # Ignore generic terms
            if "given" not in raw_ent:
                intent_metadata["filters"]["PortfolioName"] = raw_ent
        return intent_metadata
        
    # --- 3. Direct Fact Questions ---
    # Keyword: "custodian", "date", "security"
    if "custodian" in q_lower:
        intent_metadata["intent"] = "FACT"
        intent_metadata["target_col"] = "CustodianName"
        
        match = re.search(r"of\s+(?:the\s+|a\s+)?([\w\s\-]+?)(?:$|\?)", q_lower)
        if match:
            raw_ent = match.group(1).strip()
            intent_metadata["filters"]["PortfolioName"] = raw_ent
        return intent_metadata
        
    return intent_metadata


## 3. Query Execution Engine
Executes the parsed intent against the Pandas DataFrames with strict error handling.

In [ ]:
def execute_query(metadata, h_df, t_df):
    # 1. Select DataFrame
    df = t_df if metadata["table"] == "trades" else h_df
    if df is None: return "Sorry can not find the answer"
    
    result_df = df.copy()
    
    # 2. Apply Filters
    for col, val in metadata["filters"].items():
        if col not in result_df.columns:
            return "Sorry can not find the answer"
        
        mask = result_df[col].astype(str).str.contains(val, case=False, regex=False)
        result_df = result_df[mask]
        
    if result_df.empty:
        return "Sorry can not find the answer"
        
    # 3. Apply Logic based on Intent
    intent = metadata.get("intent")
    
    try:
        if intent == "PERFORMANCE":
            # Which fund performed better (PL_YTD)
            if "PL_YTD" not in result_df.columns or "PortfolioName" not in result_df.columns:
                 return "Sorry can not find the answer"
            
            grouped = result_df.groupby("PortfolioName")["PL_YTD"].sum().reset_index()
            grouped = grouped.sort_values(by="PL_YTD", ascending=False)
            
            if grouped.empty:
                return "Sorry can not find the answer"
                
            best_fund = grouped.iloc[0]["PortfolioName"]
            return f"{best_fund}"
            
        elif intent == "AGGREGATION":
            count = len(result_df)
            return f"{count}"
            
        elif intent == "FACT":
            target = metadata.get("target_col")
            if target and target in result_df.columns:
                answers = result_df[target].unique()
                return ", ".join(map(str, answers))
            else:
                return "Sorry can not find the answer"
                
    except Exception:
        return "Sorry can not find the answer"
        
    return "Sorry can not find the answer"

## 4. Main Chat Interface

In [ ]:
def chat(question):
    # 1. Parse
    metadata = classify_and_parse(question)
    if metadata["intent"] == "UNKNOWN":
        return "Sorry can not find the answer"
        
    # 2. Query
    answer = execute_query(metadata, h_df, t_df)
    return answer

## 5. Sample Interactions & Verification

In [ ]:
test_questions = [
    # Valid Questions
    "Total number of trades for Platpot Fund",
    "Total number of holdings for Garfield",
    "Which funds performed better depending on the yearly Profit & Loss of that fund",
    "Custodian of Heather",
    
    # Edge Cases / Invalid
    "Total number of trades for NonExistentFund",
    "What is the weather today?",
    "Custodian of NonExistentFund"
]

print("{:<80} | {:<30}".format("Question", "Answer"))
print("-"*110)
for q in test_questions:
    ans = chat(q)
    print("{:<80} | {:<30}".format(q, str(ans)))